In [3]:
import pandas as pd

# Parquet 파일 경로
parquet_file_path = r'C:\github\MathBridge\spoken_LaTex\Data\df_not_len_5_cleaned_unique_eq.parquet'

# Parquet 파일의 첫 몇 개의 행을 불러오기
df_head = pd.read_parquet(parquet_file_path, engine='pyarrow')

print(df_head.head())   # 데이터의 첫 몇 개의 행을 출력하여 구조 확인
print(df_head.info())   # 데이터의 기본 정보(열 이름, 데이터 타입 등)를 확인



                                      context_before           equation  \
0  The horizontal axis represents the exponent range  $ \in [ 3 , 7 ] $   
1                                               Here          $ d\eta $   
2                               The agent 's actions        $ a ( t ) $   
3  are modeled by a stochastic control process wi...     $ \sigma^2_t $   
4                                               Here        $ \deltat $   

                                       context_after  eq_type paper_number  \
0  . We selected those categorical colors from Co...        0   2301.00002   
1  denotes the system noise , modeled as a Wiener...        0   2301.00005   
2  are modeled by a stochastic control process wi...        0   2301.00005   
3  controlled by the agent and with a mean of zer...        0   2301.00005   
4  denotes the physical time step , and we adopte...        0   2301.00005   

  paper_type  equation_len  
0  arxiv2023            17  
1  arxiv2023          

In [4]:
import pandas as pd
import re
import json
import time
from tqdm import tqdm

def load_patterns(filename):
    """JSON 파일에서 패턴 로드"""
    with open(filename, 'r') as file:
        return json.load(file)

def preprocess_latex(latex_str):
    """특정 기호 앞에 공백 추가 및 필요시 공백 제거"""
    # 특정 기호 (\ ^ &) 앞에 공백 추가
    latex_str = re.sub(r'(?<!\s)([\\^&])', r' \1', latex_str)
    
    # \ 앞뒤로 공백만 있을 경우 공백 제거
    latex_str = re.sub(r'\s*\\\s*', r'\\', latex_str)

    # LaTeX 명령어와 기호를 공백으로 분리
    latex_str = re.sub(r'(\\[A-Za-z]+)([^A-Za-z\s])', r'\1 \2', latex_str)
    latex_str = re.sub(r'([^A-Za-z\s])(\\[A-Za-z]+)', r'\1 \2', latex_str)
    
    return latex_str

def replace_symbols_with_words(latex_str):
    """기호를 자연어로 변환하고 앞뒤에 공백 추가"""
    replacements = {
        '+': ' plus ',
        '-': ' minus ',
        '/': ' divided by ',
        '_': ' subscript ',
        '^': ' to the power of ',
        '=': ' equals ',
        '<': ' less than ',
        '>': ' greater than ',
        '(': ' open parenthesis ',
        ')': ' close parenthesis ',
        '[': ' open bracket ',
        ']': ' close bracket ',
        '{': ' open brace ',
        '}': ' close brace ',
        '&': ' and ',
        '%': ' percent ',
        '*': ' times ',
        ':': ' colon ',
        ';': ' semicolon ',
        ',': ' comma ',
        '.': ' point ',
        '|': ' vertical bar ',
        '\\': ' backslash ',
        '!': ' factorial ',
        '@': ' at ',
        '#': ' hash ',
        '$': ' dollar ',
        '~': ' tilde ',
        '`': ' backtick ',
        '\'': ' single quote ',
        '\"': ' double quote ',
        '?': ' question mark ',
        '°': ' degree ',
        '∞': ' infinity ',
        'π': ' pi ',
        '∑': ' summation ',
        '∏': ' product ',
        '√': ' square root ',
        '∫': ' integral ',
        '∂': ' partial derivative ',
        '∆': ' delta ',
        '∇': ' nabla ',
        '⊗': ' tensor product ',
        '⊕': ' direct sum ',
        '⊆': ' subset ',
        '⊂': ' proper subset ',
        '∪': ' union ',
        '∩': ' intersection ',
        '≠': ' not equal ',
        '≈': ' approximately equal ',
        '≡': ' identical to ',
        '∈': ' element of ',
        '∉': ' not an element of ',
        '⊂': ' subset of ',
        '⊃': ' superset of ',
        '∪': ' union ',
        '∩': ' intersection ',
        'ℵ': ' aleph ',
        'ℶ': ' beth ',
        'ℷ': ' gimel ',
        'ℸ': ' daleth ',
        '→': ' right arrow ',
        '←': ' left arrow ',
        '↑': ' up arrow ',
        '↓': ' down arrow ',
        '↔': ' left-right arrow ',
        '⇔': ' if and only if ',
    }

    # 기호를 자연어로 변환하고 앞뒤에 공백 추가
    for symbol, word in replacements.items():
        latex_str = latex_str.replace(symbol, f' {word} ')
    
    return latex_str

def apply_patterns(latex_str, patterns):
    """패턴을 적용하여 LaTeX 수식을 자연어로 변환"""
    latex_str = preprocess_latex(latex_str)
    latex_str = replace_symbols_with_words(latex_str)  # 기호를 먼저 변환
    
    for pattern in patterns:
        regex_pattern = pattern['pattern']
        replacement = pattern['replacement']
        
        # 정규식에서 \ 를 이스케이프 처리
        regex_pattern = re.escape(regex_pattern)
        
        latex_str = re.sub(regex_pattern, lambda m: eval(f'f"{replacement}"'), latex_str)
    
    # 변환 후 중복된 공백 제거
    latex_str = re.sub(r'\s+', ' ', latex_str).strip()
    
    return latex_str

def latex_to_english(latex_str, patterns):
    """LaTeX 수식을 자연어로 변환"""
    latex_str = latex_str.replace(r'\frac', 'frac')
    latex_str = latex_str.replace('$', '')  # $ 기호 삭제
    latex_str = apply_patterns(latex_str, patterns)
    return latex_str

# JSON 파일에서 패턴 로드
patterns = load_patterns('patterns.json')

# Parquet 파일 경로
parquet_file_path = r'C:\github\MathBridge\spoken_LaTex\Data\df_not_len_5_cleaned_unique_eq.parquet'

# Parquet 파일의 데이터를 한 번에 불러오기
df = pd.read_parquet(parquet_file_path, engine='pyarrow')

# 랜덤으로 1000개의 데이터를 샘플링
df_sampled = df.sample(n=1000, random_state=42)

# 변환 작업 전 시간 기록
start_time = time.time()

# tqdm을 사용하여 진행률 표시 및 로그 추가
equation_en = []
complete = []

for equation in tqdm(df_sampled['equation'], desc="Processing equations", unit="equations"):
    en_equation = latex_to_english(equation, patterns)
    equation_en.append(en_equation)
    # \ 기호가 있는지 확인하여 complete 항목 생성
    complete.append(0 if '\\' in en_equation else 1)

# 새로운 열에 변환 결과 및 complete 항목 추가
df_sampled['equation_en'] = equation_en
df_sampled['complete'] = complete

# complete 항목을 기준으로 0과 1로 데이터셋 분리
df_complete_1 = df_sampled[df_sampled['complete'] == 1]
df_complete_0 = df_sampled[df_sampled['complete'] == 0]

# 비율 계산
complete_1_ratio = len(df_complete_1) / len(df_sampled) * 100
complete_0_ratio = len(df_complete_0) / len(df_sampled) * 100

# 변환 작업 후 시간 기록 및 처리 속도 계산
end_time = time.time()
elapsed_time = end_time - start_time
speed = len(df_sampled) / elapsed_time

# 로그 출력
print(f"Transformation complete and saved to parquet files")
print(f"Total time taken: {elapsed_time:.2f} seconds")
print(f"Processing speed: {speed:.2f} equations per second")
print(f"Complete 1 ratio: {complete_1_ratio:.2f}%")
print(f"Complete 0 ratio: {complete_0_ratio:.2f}%")


Processing equations: 100%|██████████| 1000/1000 [00:00<00:00, 4886.51equations/s]

Transformation complete and saved to parquet files
Total time taken: 0.27 seconds
Processing speed: 3660.05 equations per second
Complete 1 ratio: 100.00%
Complete 0 ratio: 0.00%


In [7]:
import pandas as pd

# Parquet 파일 경로
parquet_file_path = r'C:\github\MathBridge\spoken_LaTex\Data\df_not_len_5_cleaned_unique_eq_en.parquet'

# Parquet 파일의 첫 몇 개의 행을 불러오기
df_head = pd.read_parquet(parquet_file_path, engine='pyarrow')

print(df_head.head())   # 데이터의 첫 몇 개의 행을 출력하여 구조 확인
print(df_head.info())   # 데이터의 기본 정보(열 이름, 데이터 타입 등)를 확인

                                      context_before           equation  \
0  The horizontal axis represents the exponent range  $ \in [ 3 , 7 ] $   
1                                               Here          $ d\eta $   
2                               The agent 's actions        $ a ( t ) $   
3  are modeled by a stochastic control process wi...     $ \sigma^2_t $   
4                                               Here        $ \deltat $   

                                       context_after  eq_type paper_number  \
0  . We selected those categorical colors from Co...        0   2301.00002   
1  denotes the system noise , modeled as a Wiener...        0   2301.00005   
2  are modeled by a stochastic control process wi...        0   2301.00005   
3  controlled by the agent and with a mean of zer...        0   2301.00005   
4  denotes the physical time step , and we adopte...        0   2301.00005   

  paper_type  equation_len                               equation_en  
0  arxiv2